# Model Evaluation
- Evaluate the best Model by choosing between different Model types, Tuning parameters, and selecting Features
- Select the right **Model evaluation metric** to quantify the Model performance

<hr>

## Model Evaluation Procedures:
1. Train/Test Split:
    - Good estimate of out-of-sample performance, but with high variance
    - Fast, simple, and flexible
2. K-fold Cross-Validation:
    - Better estimate of out-of-sample performance
    - "K" times slower than Train/Test split

## Model Evaluation Metric:
1. Regression problems:
    - Mean Absolute Error (MAE)
    - Mean Squared Error (MSE)
    - Root Mean Squared Error (RMSE)
2. Classification problems:
    - Classification Accuracy

<hr>

# Load Data and Modules
- Using locally saved copy of **Pima Indian Diabetes** dataset from the *UCI Machine Learning Repository* from kaggle

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read the data into a Pandas DataFrame
pima = pd.read_csv("Data/diabetes.csv")
pima.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# Change the names of the columns
pima.columns = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
pima.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
pima.shape

(768, 9)

- Label is used to indicate if the patient has diabetes (1) or no diabetes (0)
- Predict diabetes status given patient's health measurements

<hr>

# Classification Accuracy
- Using **Classification Accuracy** as evaluation metric
- Using **Train/Test Split** as evaluation procedure

## 1. Get the data and define X and y
- For Features using only **pregnant**, **insulin**, **bmi**, and **age** Variables

In [5]:
# Define X and y
feature_cols = ['pregnant', 'insulin', 'bmi', 'age']
X = pima[feature_cols]
y = pima.label

## 2. Data Partition using Train/Test Split

In [6]:
# Import the module to Split the data into Training and Testing
from sklearn.model_selection import train_test_split

# Split by 25% Testing, 75% Training (default parameters)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## 3. Train the Model on the Training Set

In [7]:
# Import the class module
from sklearn.linear_model import LogisticRegression

# Initiate the Model
logreg = LogisticRegression()

# Fit the Model on Training Set
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## 4. Predict on the Testing Set

In [8]:
# Predict on new testing data
y_pred = logreg.predict(X_test)

## 5. Classification Accuracy: percentage of correct predictions

In [9]:
# Import metrics module to check accuracy
from sklearn import metrics

metrics.accuracy_score(y_test, y_pred)

0.69270833333333337

- Classification Accuracy of 69%

### Comapre to Benchmark Accuracy - *Null Accuracy*
- Predict the most frequent class

In [10]:
# Look at the distribution of the testing set 
y_test.value_counts()

0    130
1     62
Name: label, dtype: int64

In [11]:
# Calculate the precentage of ones
y_test.mean()  # Same as 62/192

0.32291666666666669

In [12]:
# Calculate the precentage of zeros
1 - y_test.mean()

0.67708333333333326

- 68% is the Null Accuracy for this problem.  Beanchmark baseline Accuracy is at 68%

In [13]:
# Claculate Null Accuracy for Binary Classification in one step
max(y_test.mean(), 1 - y_test.mean())

0.67708333333333326

In [14]:
# Claculate Null Accuracy for Multi-class Classification in one step (only works for Pandas Series)
y_test.value_counts().head(1) / len(y_test)

0    0.677083
Name: label, dtype: float64

#### Conclusion:
- **Classification Accuracy** is an easy metric to understand
- However, it does not tell the underlying distribution of response values, and the types of errors

<hr>

# Confusion Matrix
- 2x2 NumPy array that describes the performance of a classification model

In [15]:
# First argument has to the the ACTUAL values, second argument is the PREDICTED values
metrics.confusion_matrix(y_test, y_pred)

array([[118,  12],
       [ 47,  15]])

|                 |  Predicted 0:    |  Predicted 1:     |
|-----------------|------------------|                   |
|  **Actual 0:**  |  118             |  12               | 
|  **Actual 1:**  |  47              |  15               |

- **True Positive (TP)**: *correctly* predicted *positive*
- **True Negative (TN)**: *correctly* predicted *negative*
- **False Positive (FP)**: *incorrectly* predicted *positive* - "Type I Error"
- **False Negative (FN)**: *incorrectly* predicted *negative* - "Type II Error"

|       |  (-)  |  (+)  |
|-------|---------------|
|  **(-)**  |  TN   |  FP   | 
|  **(+)**  |  FN   |  TP   | 

In [16]:
# Save Confusion Matrix values 
confusion = metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]

## Metrics from Confusion Matrix

### Classification Accuracy: How accurate is the classifier

In [18]:
print((TP + TN) / (TP + FP + FN + TN))
print(metrics.accuracy_score(y_test, y_pred))

0.692708333333
0.692708333333


### Classification Error: How often is the classifier incorrect

In [19]:
print((FP + FN) / (TP + TN + FP + FN))
print(1 - metrics.accuracy_score(y_test, y_pred))

0.307291666667
0.307291666667


### Sensitivity: P(Positive Test | Disease): Also known as *"Recall"* and *"True Positive Rate"*

In [20]:
print(TP / (TP + FN))
print(metrics.recall_score(y_test, y_pred))

0.241935483871
0.241935483871


### Specificity: P(Negative Test | No Disease): How *selective* is the classifier in predicting positive instances

In [21]:
print(TN / (FP + TN))

0.907692307692


### False Positive Rate: When Actual value is *Negative*, how often is the prediction incorrect
- 1 - Specificity

In [24]:
print(FP / (TN + FP))

0.0923076923077


### Positive Predictive Value: P(Disease | Positive Test) also known as *Precision*

In [22]:
print(TP / (TP + FP))

0.555555555556


### Negative Predictive Value: P(No Disease | Negative Test)

In [23]:
print(TN / (FN + TN))

0.715151515152


## Which metric to choose:
- **Spam filter**: Optimize **Precision** or **Specificity** because *false negative* are more acceptable than *false positive*
- **Fraud detector**: Optimize for **Sensitivity**  because *false positive* are more acceptable than *false negatives*

<hr>